In [25]:
# Configuration(Need to setting)
project_home_path = "/Users/yoonseojin/Statistics_sj"
left_hand_data_dir_path = project_home_path + "/CLMN/Replay_Exp/experiment/20210325_blueprint_0324v2/left hand"  # Left Hand
participant_name = "jonghyuk"

import os
os.chdir(project_home_path)

import pandas as pd
pd.set_option("display.width", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

import numpy as np
import matplotlib.pyplot as plt

from CLMN.Replay_Exp.experiment.Replay_Experiment_Tool import mapping_data_current, single_response_only
from File_Package.sj_file_system import CsvManager

seq1 = ["4","1","3","2","4"]
seq2 = ["1","4","2","3","1"]
seq1_name = "['4', '1', '3', '2', '4']"
seq2_name = "['1', '4', '2', '3', '1']"
seq1_color = "blue"
seq2_color = "red"

def find_index(previous_datas, target_seq):
    """
    :param datas: 이전 데이터들
    :param target_seq: 입력해야 하는 sequence
    
    return: {
        previous_correct_index: 이전의 맞은 index,
        current_target_index: 현재 입력해야 하는 index
    }
    """
    
    correct_previous_data_index = -1
    current_target_index = 0
    for i in range(0, len(previous_datas)):
        if previous_datas.iloc[i]["Response"] == target_seq[current_target_index]:
            correct_previous_data_index = i
            current_target_index += 1
            
        if current_target_index >= len(target_seq):
            # 한 시퀀스를 모두 다 찾았을때, index 초기화
            current_target_index = 0
            correct_previous_data_index = -1
            
    return {
        "previous_correct_index" : correct_previous_data_index,
        "current_target_index" : current_target_index
    }

In [2]:
"""
Load Data
"""
stimuluses = []
responses = []
for i in range(0, 4):
    stimulus_file_name = "stimulus_" + participant_name + "_" + str(i)
    response_file_name = "response_" + participant_name + "_" + str(i)
    stimuluses.append( CsvManager(dir_path=left_hand_data_dir_path, file_name=stimulus_file_name).read_csv_from_pandas()[2:-1] ) # remove not necessary data
    responses.append( CsvManager(dir_path=left_hand_data_dir_path, file_name=response_file_name).read_csv_from_pandas() )
stimuluses[-1] = stimuluses[-1][:-1] # remove not necessary data


In [3]:
mapped_d = mapping_data_current(stimuluses, responses)
mapped_d = single_response_only(mapped_d)

In [26]:
mapped_datas = mapped_d
response_times_across_runs = []
for run_index in range(0, len(mapped_datas)):
    response_times_per_run = []
    # groupby를 step으로 처리
    for step_name, step_group in mapped_datas[run_index].groupby("Step"):
        # intertap interval 정의
        # 한 스텝에서 처음 시퀀스가 완료된 경우: 현재 반응시간 - 자극 시간
        # 그 외의 경우: 현재 반응 시간 - 이전의 맞는 입력의 반응시간
        response_times_per_step = []
        for group_row_index in range(0, len(step_group)):
            current_row = step_group.iloc[group_row_index]
            target_seq = []
            if current_row["Stimulus"] == seq1_name:
                target_seq = seq1
            else:
                target_seq = seq2
            
            # 이전에 있는 입력들중 맞는 입력이 있는지 체크
            # 만약 맞는 입력이 없다면 stimulus의 자극 시간과 비교
            find_result = find_index(previous_datas = step_group[0:group_row_index], 
                                     target_seq = target_seq)
            previous_correct_index = find_result["previous_correct_index"]
            current_target_index = find_result["current_target_index"]
            
            if previous_correct_index == -1:
                continue
            else:
                previous_correct_row = step_group.iloc[previous_correct_index]
                if target_seq[current_target_index] == current_row["Response"]:
                    response_times_per_step.append([(previous_correct_row["Response"], current_row["Response"]), current_row["response_seconds"] - previous_correct_row["response_seconds"]])
        response_times_per_run.append({
            "Step": step_name,
            "Response_times": response_times_per_step
        })
    response_times_across_runs.append(response_times_per_run)



In [31]:
response_times_across_runs[0][0]



'\nrun 0에 대하여,\n\nX축은 step\n\n\n'

In [32]:
"""
run 0에 대하여,

X축은 step
Y축은 intertap interval
"""

print(100)

100
